## Notebook for dataset generation for AE4317 Autonomous Flight of Micro Air Vehicles

Some prerequisites to run this notebook:
- Depth Anything
- Blabla store all real images in /images_real and all simulator images in /images_sim

### Imports

In [1]:
from data_functions import *

### Convert images to depth maps
We have chosen to use Depth Anything (see [here](https://github.com/LiheYoung/Depth-Anything)) to create depth maps of the images. The depth maps are then used to classify each image as either 'left', 'forward' or 'right', corresponding to the control action to take for the given image.

Note: it takes quite some time to convert a large number of images to depth maps, consider using a GPU

In [3]:
real_images_folder = 'real_images'
sim_images_folder = 'sim_images'

# generate depth maps
# generate_depth_images()

### Create labels
Based on the generated depth maps the images are classified

In [ ]:
all_images_folder = 'all_images'
labeled_images = pd.DataFrame(columns=['filename', 'left', 'forward', 'right'])

labeled_images = generate_labels(real_images_folder, all_images_folder, labeled_images, mirror=True)
labeled_images = generate_labels(sim_images_folder, all_images_folder, labeled_images, mirror=True, make_realistic=True)

labeled_images.to_csv('labeled_images.csv', index=False)

Now we have all the ingredients we need to train our CNN, time to hop over to the next notebook!